In [1]:
import numpy as np
import geopandas as gpd
import codecs

In [52]:
########### USER ###########
#set domain
domain = 'WY'

In [53]:
# path to DEM ascii
DEMpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/DEM_'+domain+'.asc'

#path to SNOTEL gdf
gdfpath = '/nfs/attic/dfh/Aragon2/CSOdata/'+domain+'/CSO_SNOTEL_sites.geojson'

#path to VEG .asc
VEGpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/NLCD2016_'+domain+'.asc'

# outfile path
outVEGpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/NLCD2016_line_'+domain+'.asc'

# outfile path
outDEMpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/DEM_line_'+domain+'.asc'

# outfile path
outFpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/snowmodel_line_pts.dat'

# path to SnowModel
modpath = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/'

# first run the Fetch_SNOTEL notebook to find all of the stations within the modeling domain
# this will save lat lon coordinates of stations to projected cordinates that match the DEM

stn_gdf = gpd.read_file(gdfpath)

In [54]:
def getmeta(file_name):
    myvars = {}
    lines = open(file_name, 'r').readlines()
    for i in range(5):
        line=lines[i]
        name, var = line.partition("\t")[::2]
        myvars[name.strip()] = int(var)
    return myvars

In [55]:
myvars = getmeta(DEMpath)
myvars

{'ncols': 1382,
 'nrows': 2476,
 'xllcorner': 487200,
 'yllcorner': 4690100,
 'cellsize': 100}

In [56]:
#Build northing easting array 
#Northing
# locate center of lower left cell
st = myvars['yllcorner']+(myvars['cellsize']/2)
#build northing array
north = np.flip(np.arange(st,st+myvars['nrows']*myvars['cellsize'],myvars['cellsize']),0)

#Easting
# locate center of lower left cell
st = myvars['xllcorner']+(myvars['cellsize']/2)
#build easting array
east = np.arange(st,st+myvars['ncols']*myvars['cellsize'],myvars['cellsize'])

In [57]:
#fortran indexing starts at 1
#staion 1 to N of N stations
count = np.zeros(stn_gdf.shape[0])
#easting of pixel corresponding to station 
stn_est = np.zeros(stn_gdf.shape[0])
#northing of pixel corresponding to station
stn_nor = np.zeros(stn_gdf.shape[0])
#index of pixel corresponding to station 
est_idx = np.zeros(stn_gdf.shape[0])
#index of pixel corresponding to station 
nor_idx = np.zeros(stn_gdf.shape[0])

for z in range(stn_gdf.shape[0]):
    count[z] = z + 1
    lons = abs(stn_gdf.easting[z]-east)
    loIDX = [i for i, value in enumerate(lons) if value == np.min(abs(stn_gdf.easting[z]-east))] 
    stn_est[z] = east[loIDX[0]]
    est_idx[z] = loIDX[0] + 1
    lats = abs(stn_gdf.northing[z]-north)
    laIDX = [i for i, value in enumerate(lats) if value == np.min(abs(stn_gdf.northing[z]-north))]
    stn_nor[z] = north[laIDX[0]]
    nor_idx[z] = laIDX[0] + 1

In [58]:
print(count)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]


In [59]:
#Print out .dat file 
#move this file to extra_met directory in SnowModel code
#https://pyformat.info/
f= open(outFpath,"w+")
for z in range(count.shape[0]):
    f.write('{:08.0f}\t'.format(count[z])+'{:10.0f}\t'.format(est_idx[z])+'{:10.0f}\t'.format(nor_idx[z])+
            '{:10.0f}\t'.format(stn_est[z])+'{:10.0f}\t\n'.format(stn_nor[z]))
f.close() 

# Extract topo and veg files for snowmodel_line

In [60]:
#dem
with codecs.open(DEMpath, encoding='utf-8-sig') as f:
    dem = np.loadtxt(f,skiprows=6)

#veg
with codecs.open(VEGpath, encoding='utf-8-sig') as f:
    veg = np.loadtxt(f,skiprows=6)

In [61]:
dem_line=[]
veg_line=[]
for i in range(count.shape[0]):
    dem_line.append(str(int(dem[int(nor_idx[i]-1),int(est_idx[i]-1)])))
    veg_line.append(str(int(veg[int(nor_idx[i]-1),int(est_idx[i]-1)])))

In [62]:
lines = open(DEMpath, 'r').readlines()
head = 'ncols\t1\nnrows\t'+str(count.shape[0])+'\n'+lines[2]+lines[3]+lines[4]+lines[5]

data = ''
for s in dem_line:
    data += s +'\n'

f= open(outDEMpath,"w+")
f.write(head+data)
f.close()

In [51]:
lines = open(VEGpath, 'r').readlines()
head = 'ncols\t1\nnrows\t'+str(count.shape[0])+'\n'+lines[2]+lines[3]+lines[4]+lines[5]

data = ''
for s in veg_line:
    data += s +'\n'

f= open(outVEGpath,"w+")
f.write(head+data)
f.close()